<a href="https://colab.research.google.com/github/twice1st/Detecting-and-Beautifying-Vietnamese-Faces/blob/main/Source%20code/Embedding_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np

In [ ]:
len(os.listdir("/content/drive/MyDrive/CS231/data_all/"))

19589

In [ ]:
path_emb_sm2 = "/content/drive/MyDrive/CS231/nn4.small2.v1.t7"
path_emb_sm1 = '/content/drive/MyDrive/CS231/nn4.small1.v1.t7'
path_emb_v2 = '/content/drive/MyDrive/CS231/nn4.v2.t7'
def _load_torch(model_path_fl):
  """
  model_path_fl: Link file chứa weigth của model
  """
  model = cv2.dnn.readNetFromTorch(model_path_fl)
  return model

encoder_sm1 = _load_torch(path_emb_sm1)
encoder_sm2 = _load_torch(path_emb_sm2)
encoder_v2  = _load_torch(path_emb_v2)

In [ ]:
# import cv2

def _blobImage(image, out_size = (300, 300), scaleFactor = 1.0, mean = (104.0, 177.0, 123.0)):
  """
  input:
    image: ma trận RGB của ảnh input
    out_size: kích thước ảnh blob
  return:
    imageBlob: ảnh blob
  """
  # Chuyển sang blobImage để tránh ảnh bị nhiễu sáng
  imageBlob = cv2.dnn.blobFromImage(image, 
                                    scalefactor=scaleFactor,   # Scale image
                                    size=out_size,  # Output shape
                                    mean=mean,  # Trung bình kênh theo RGB
                                    swapRB=False,  # Trường hợp ảnh là BGR thì set bằng True để chuyển qua RGB
                                    crop=False)
  return imageBlob

In [ ]:
def _image_read(image_path):
  """
  input:
    image_path: link file ảnh
  return:
    image: numpy array của ảnh
  """
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image

In [ ]:
import pickle

def _save_pickle(obj, file_path):
  with open(file_path, 'wb') as f:
    pickle.dump(obj, f)

def _load_pickle(file_path):
  with open(file_path, 'rb') as f:
    obj = pickle.load(f)
  return obj

In [ ]:
_save_pickle(faces, "/content/drive/MyDrive/CS231/file/faces.pkl")
_save_pickle(labels, "/content/drive/MyDrive/CS231/file/labels.pkl")
_save_pickle(img_paths, "/content/drive/MyDrive/CS231/file/images_file.pkl")

In [ ]:
faces = _load_pickle("/content/drive/MyDrive/CS231/file/Trong/faces.pkl")
labels = _load_pickle("/content/drive/MyDrive/CS231/file/Trong/labels.pkl")
img_paths = _load_pickle("/content/drive/MyDrive/CS231/file/Trong/images_file.pkl")

In [ ]:
def _embedding_faces(encoder, faces):
  emb_vecs = []
  for face in faces:
    faceBlob = _blobImage(face, out_size = (96, 96), scaleFactor=1/255.0, mean=(0, 0, 0))
    # Embedding face
    encoder.setInput(faceBlob)
    vec = encoder.forward()
    emb_vecs.append(vec)
  return emb_vecs

embed_faces_sm1 = _embedding_faces(encoder_sm1, faces)
embed_faces_sm2 = _embedding_faces(encoder_sm2, faces)
embed_faces_v2 = _embedding_faces(encoder_v2, faces)
# Nhớ save embed_faces vào Dataset.
_save_pickle(embed_faces_sm1, "/content/drive/MyDrive/CS231/file/Trong/embed_blob_faces_sm1_300.pkl")
_save_pickle(embed_faces_sm2, "/content/drive/MyDrive/CS231/file/Trong/embed_blob_faces_sm2_300.pkl")
_save_pickle(embed_faces_v2, "/content/drive/MyDrive/CS231/file/Trong/embed_blob_faces_v2_300.pkl")

In [ ]:
from google.colab.patches import cv2_imshow
import cv2


In [ ]:
def _embedding_faces(encoder, faces):
  emb_vecs = []
  for face in faces:
    # faceBlob = _blobImage(face, out_size = (96, 96), scaleFactor=1/255.0, mean=(0, 0, 0))
    # Embedding face
    face_resize = cv2.resize(face, (96,96))/255
    face_resize = face_resize.reshape(1,3,96,96)
    encoder.setInput(face_resize)
    vec = encoder.forward()
    emb_vecs.append(vec)
  return emb_vecs

embed_faces_sm1 = _embedding_faces(encoder_sm1, faces)
embed_faces_sm2 = _embedding_faces(encoder_sm2, faces)
embed_faces_v2 = _embedding_faces(encoder_v2, faces)
# Nhớ save embed_faces vào Dataset.
_save_pickle(embed_faces_sm1, "/content/drive/MyDrive/CS231/file/Trong/embed_blob_faces_sm1_no_processing.pkl")
_save_pickle(embed_faces_sm2, "/content/drive/MyDrive/CS231/file/Trong/embed_blob_faces_sm2_no_processing.pkl")
_save_pickle(embed_faces_v2, "/content/drive/MyDrive/CS231/file/Trong/embed_blob_faces_v2_no_processing.pkl")

In [ ]:
from sklearn.model_selection import train_test_split
ids = np.arange(len(labels))
X_train, X_test, y_train, y_test, id_train, id_test = train_test_split(np.stack(embed_faces), labels, ids, test_size = 0.2, stratify = labels)
X_train = np.squeeze(X_train, axis = 1)
X_test = np.squeeze(X_test, axis = 1)
print(X_train.shape, X_test.shape)
print(len(y_train), len(y_test))

(15671, 128) (3918, 128)
15671 3918


In [ ]:
y_test[1]

'1'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def _most_similarity(embed_vecs, vec, labels):
  sim = cosine_similarity(embed_vecs, vec)
  sim = np.squeeze(sim, axis = 1)
  argmax = np.argsort(sim)[::-1][:1]
  label = [labels[idx] for idx in argmax][0]
  return label

# Lấy ngẫu nhiên một bức ảnh trong test
vec = X_test[1].reshape(1, -1)
# Tìm kiếm ảnh gần nhất
_most_similarity(X_train, vec, y_train)

'1'

# sm2

In [ ]:
# def _acc_test(test_set, y_test):
from sklearn.metrics import accuracy_score

y_preds = []
for vec in X_test:
  vec = vec.reshape(1, -1)
  y_pred = _most_similarity(X_train, vec, y_train)
  y_preds.append(y_pred)

print(accuracy_score(y_preds, y_test))

0.7401735579377233


# sm1

In [ ]:
# def _acc_test(test_set, y_test):
from sklearn.metrics import accuracy_score

y_preds = []
for vec in X_test:
  vec = vec.reshape(1, -1)
  y_pred = _most_similarity(X_train, vec, y_train)
  y_preds.append(y_pred)

print(accuracy_score(y_preds, y_test))

0.7274119448698315


# v2

In [ ]:
# def _acc_test(test_set, y_test):
from sklearn.metrics import accuracy_score

y_preds = []
for vec in X_test:
  vec = vec.reshape(1, -1)
  y_pred = _most_similarity(X_train, vec, y_train)
  y_preds.append(y_pred)

print(accuracy_score(y_preds, y_test))

0.6763654925982644


In [2]:
print("NGK")

NGK
